In [1]:
from profess_calculator import PROFESS
import profess_io as pio
from cpmd import initialize_atoms
from ase import units
import os
# import pixiedust
import numpy as np

In [2]:
# initialize
pos_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step0.dat'
vel_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/vel_step0.dat'
cell_param = [[3.97, 0, 0], [0, 3.97, 0], [0, 0, 3.97]]
atoms = initialize_atoms(pos_file, vel_file, cell_param, pos_type='FRAC')
# set parameters for PROFESS calculation
run_dir = '/home/misa/git_repositories/PROFESS/test/ase_singlepoint'
inpt_name = 'sp'
pp_names = ['al_HC.lda.recpot']
# print('ID of atoms', id(atoms))
calc = PROFESS(run_dir, inpt_name, pp_names, atoms)
print('ID of calc.atoms is equal to ID of atoms:', id(calc.atoms)==id(atoms) )
atoms.calc = calc
print('ID of calc.atoms is equal to ID of atoms:', id(calc.atoms)==id(atoms) )

ID of calc.atoms is equal to ID of atoms: True
ID of calc.atoms is equal to ID of atoms: False


every object in python has a unique id
atoms = initialize_atoms(pos_file, vel_file, cell_param, pos_type='FRAC') -> atoms has an id
PROFESS(...) the atoms object in PROFESS has the same id because it is a reference

atoms.calc = PROFESS()

now the ID of the atoms object has to change because otherwise an infinite generation of calculators, atom pairs is triggered:

if atoms has a calculator, calc.atoms would also need a calculator (like calc.atoms.calc) but this would change atoms again to atoms.calc.atoms.calc, which would then change the atoms object in atoms.calc.atoms and so on

This issue can be solved by adding an empty calculator to the atoms object in the first step

Then calc.atoms is changed from None to calc.atoms = atoms

This gives the desired behaviour (a change of atoms.positions also causes a change of atoms.calc.positions but it is still a little bit strange because now the object ID of atoms belongs to atoms which has calc as an attribute and to atoms.calc.atoms which is an attribute of calc (and also of itself)

In [3]:
# initialize
pos_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/ion_step0.dat'
vel_file = '/home/misa/git_repositories/PROFESS/test/MD_NVE/saved/vel_step0.dat'
cell_param = [[3.97, 0, 0], [0, 3.97, 0], [0, 0, 3.97]]
atoms = initialize_atoms(pos_file, vel_file, cell_param, pos_type='FRAC')

# set parameters for PROFESS calculation
run_dir = '/home/misa/git_repositories/PROFESS/test/ase_singlepoint'
inpt_name = 'sp'
pp_names = ['al_HC.lda.recpot']

# make empty calculator
calc = PROFESS()
# pass calculator to atoms object
atoms.calc = calc
atoms.calc.initialize(run_dir, inpt_name, pp_names, atoms)
print('ID of calc.atoms is equal to ID of atoms:', id(calc.atoms)==id(atoms) )

ID of calc.atoms is equal to ID of atoms: True


In [29]:
atoms.get_positions()

array([[0.99253956, 0.89124982, 0.98606693],
       [2.98958857, 3.26704632, 0.99976539],
       [0.99244286, 2.89935623, 3.01363036],
       [2.965429  , 0.88234755, 2.94053733]])

In [30]:
atoms.calc.atoms.get_positions()

array([[0.99253956, 0.89124982, 0.98606693],
       [2.98958857, 3.26704632, 0.99976539],
       [0.99244286, 2.89935623, 3.01363036],
       [2.965429  , 0.88234755, 2.94053733]])

In [31]:
atoms.set_positions(atoms.get_positions()*2)
atoms.get_positions()

array([[1.98507913, 1.78249963, 1.97213386],
       [5.97917713, 6.53409264, 1.99953077],
       [1.98488572, 5.79871246, 6.02726072],
       [5.930858  , 1.76469509, 5.88107466]])

In [32]:
calc.atoms.get_positions()

array([[1.98507913, 1.78249963, 1.97213386],
       [5.97917713, 6.53409264, 1.99953077],
       [1.98488572, 5.79871246, 6.02726072],
       [5.930858  , 1.76469509, 5.88107466]])